# Azure ML - Búsqueda de hiperparámetros

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Búsqueda de hiperparámetros

Hemos visto cómo seguir el entrenamiento de modelos con mlflow ejecutando scripts. Por lo que ahora vamos a ver cómo buscar la mejor combinación de hiperparámetros para un modelo.

### Definición de espacio de búsqueda

Para poder realizar la búsqueda de hiperparámetros necesitamos definir un espacio de búsqueda. Para ello usamos la librería `azure.ai.ml.sweep`, por ejemplo este sería un código para definir un espacio de búsqueda

```python
from azure.ai.ml.sweep import Choice, Normal

command_job_for_sweep = job(
    batch_size=Choice(values=[16, 32, 64]),    
    learning_rate=Normal(mu=10, sigma=3),
)
```

Las opciones de búsqueda de valores son:

 * `Choice(values=[10,20,30])`
 * `Choice(values=range(1,10))`
 * `QUniform(min_value, max_value, q)`
 * `QLogUniform(min_value, max_value, q)`
 * `QNormal(mu, sigma, q)`
 * `QLogNormal(mu, sigma, q)`
 * `Uniform(min_value, max_value)`
 * `LogUniform(min_value, max_value)`
 * `Normal(mu, sigma)`
 * `LogNormal(mu, sigma)`

### Configuración de un método de muestreo

Hay tres tipos de métodos de muestreo:

 * `Muestreo de cuadrícula`: prueba todas las combinaciones posibles
 * `Muestreo aleatorio`: Elige aleatoriamente los valores del espacio de búsqueda
 * `Muestreo aleatorio Sobol`: Agrega un valor de inicialización al muestreo aleatorio para que los resultados sean reproducibles
 * `Muestreo bayesiano`: Elige nuevos valores en función de los resultados anteriores

#### Muestreo de cuadrícula

El código sería algo así

```python
from azure.ai.ml.sweep import Choice

command_job_for_sweep = command_job(
    batch_size=Choice(values=[16, 32, 64]),
    learning_rate=Choice(values=[0.01, 0.1, 1.0]),
)

sweep_job = command_job_for_sweep.sweep(
    sampling_algorithm = "grid",
    ...
)
```

#### Muestreo aleatorio

El código sería algo así

```python
from azure.ai.ml.sweep import Normal, Uniform

command_job_for_sweep = command_job(
    batch_size=Choice(values=[16, 32, 64]),   
    learning_rate=Normal(mu=10, sigma=3),
)

sweep_job = command_job_for_sweep.sweep(
    sampling_algorithm = "random",
    ...
)
```

#### Muestreo aleatorio Sobol

El código sería algo así

```python
from azure.ai.ml.sweep import RandomSamplingAlgorithm

sweep_job = command_job_for_sweep.sweep(
    sampling_algorithm = RandomSamplingAlgorithm(seed=123, rule="sobol"),
    ...
)
```

#### Muestreo bayesiano

El código sería algo así

```python
from azure.ai.ml.sweep import Uniform, Choice

command_job_for_sweep = job(
    batch_size=Choice(values=[16, 32, 64]),    
    learning_rate=Uniform(min_value=0.05, max_value=0.1),
)

sweep_job = command_job_for_sweep.sweep(
    sampling_algorithm = "bayesian",
    ...
)
```

### Configuración del early stopping

#### Directiva de bandidos

Detiene una prueba si la métrica de rendimiento objetivo es inferior a la mejor prueba hasta el momento por un margen especificado.

El código sería algo así

```python
from azure.ai.ml.sweep import BanditPolicy

sweep_job.early_termination = BanditPolicy(
    slack_amount = 0.2, 
    delay_evaluation = 5, 
    evaluation_interval = 1
)
```

#### Directiva de mediana de detención

Detiene las pruebas cuando la métrica de rendimiento de destino es inferior a la mediana del promedio de ejecuciones para todas las pruebas

El código sería algo así

```python
from azure.ai.ml.sweep import MedianStoppingPolicy

sweep_job.early_termination = MedianStoppingPolicy(
    delay_evaluation = 5, 
    evaluation_interval = 1
)
```

#### Directiva de selección de truncamiento

Detiene el entrenamiento si el X % de ejecuciones con menor rendimiento en cada intervalo de evaluación en función del valor de truncation_percentage que especificó para X

El código sería algo así

```python
from azure.ai.ml.sweep import TruncationSelectionPolicy

sweep_job.early_termination = TruncationSelectionPolicy(
    evaluation_interval=1, 
    truncation_percentage=20, 
    delay_evaluation=4 
)
```

## Ejemplo de búsqueda de hiperparámetros

#### Búsqueda de hiperparámetros desde la interfaz gráfica

##### Ejecutar el script

Volvemos a dar al botón con el símbolo `+`, le damos a `Create new file`, en `File type` seleccionamos `Notebook (*.ipynb)` y le ponemos el nombre `run_text-clasification-mlfow-hyperparameters.ipynb`. Ahora copiamos las siguientes celdas de código

```python
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except:
    credential = InteractiveBrowserCredential()
```

```python
ml_client = MLClient.from_config(credential=credential)
```

```python
from azure.ai.ml import command
from azure.ai.ml.sweep import Choice, LogUniform

SUSCRIPTION_ID = ""
RESOURCE_GROUP = ""
WORKSPACE = ""
EXPERIMENT_NAME = "image_classification-mlflow-hyperparameter-search"

execute_command = f"python image-classification-mlflow.py --suscription_id {SUSCRIPTION_ID} --resource_group {RESOURCE_GROUP} --workspace_name {WORKSPACE} --experiment_name {EXPERIMENT_NAME} --batch_size ${{{{inputs.batch_size}}}} --epochs ${{{{inputs.epochs}}}} --learning_rate ${{{{inputs.learning_rate}}}}"
inputs = {
    "batch_size": 8,
    "epochs": 1,
    "learning_rate": 1e-2,
}
environment = "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_GUI@latest"
compute_instance = "compute-instance-GUI"
display_name = "image-classificator"
sampling_algorithm = "random"
primary_metric = "Accuracy"
goal = "Maximize"

# configure base command
job = command(
    code="./",
    command=execute_command,
    inputs=inputs,
    environment=environment,
    compute=compute_instance,
    display_name=display_name,
    experiment_name=EXPERIMENT_NAME
)

# configure sweep
command_job_for_sweep = job(
    batch_size=Choice(values=[8, 16, 32, 64]),
    epochs=Choice(values=range(1,10)),
    learning_rate=LogUniform(-6, -2),
)

# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute=compute_instance,
    sampling_algorithm=sampling_algorithm,
    primary_metric=primary_metric,
    goal=goal,
)

# set the name of the sweep job experiment
sweep_job.experiment_name=EXPERIMENT_NAME

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)
```

Rellena `SUSCRIPTION_ID`, `RESOURCE_GROUP` y `WORKSPACE` con tus valores

returned_sweep_job = ml_client.create_or_update(sweep_job)

Hemos configurado solo 4 pruebas para que no tarde mucho. Si vamos a la sección de `Jobs` en Azure ML veremos que se están ejecutando las pruebas

#### Búsqueda de hiperparámetros con el CLI de Azure ML

No he encotrado la manera de ejecutar el script mediante la CLI de Azure ML

#### Búsqueda de hiperparámetros con el SDK de Python

Para la búsqueda de hiperparámetros ejecutamos el mismo código que ejecutamos en el notebook mediante la interfaz gráfica

In [ ]:
from azure.ai.ml import command
from azure.ai.ml.sweep import Choice, LogUniform

SUSCRIPTION_ID = ""
RESOURCE_GROUP = ""
WORKSPACE = ""
EXPERIMENT_NAME = "image_classification-mlflow-hyperparameter-search"

execute_command = f"python image-classification-mlflow.py --suscription_id {SUSCRIPTION_ID} --resource_group {RESOURCE_GROUP} --workspace_name {WORKSPACE} --experiment_name {EXPERIMENT_NAME} --batch_size ${{{{inputs.batch_size}}}} --epochs ${{{{inputs.epochs}}}} --learning_rate ${{{{inputs.learning_rate}}}}"
inputs = {
    "batch_size": 8,
    "epochs": 1,
    "learning_rate": 1e-2,
}
environment = "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_GUI@latest"
compute_instance = "compute-instance-Python"
display_name = "image-classificator"
sampling_algorithm = "random"
primary_metric = "Accuracy"
goal = "Maximize"

# configure base command
job = command(
    code="./",
    command=execute_command,
    inputs=inputs,
    environment=environment,
    compute=compute_instance,
    display_name=display_name,
    experiment_name=EXPERIMENT_NAME
)

# configure sweep
command_job_for_sweep = job(
    batch_size=Choice(values=[8, 16, 32, 64]),
    epochs=Choice(values=range(1,10)),
    learning_rate=LogUniform(-6, -2),
)

# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute=compute_instance,
    sampling_algorithm=sampling_algorithm,
    primary_metric=primary_metric,
    goal=goal,
)

# set the name of the sweep job experiment
sweep_job.experiment_name=EXPERIMENT_NAME

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)


Rellena `SUSCRIPTION_ID`, `RESOURCE_GROUP` y `WORKSPACE` con tus valores

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)